In [ ]:
! pip install rasterio tensorflow scikit-learn tqdm

import os
import numpy as np
import rasterio
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
# Define month-year keys
time_keys = ['2022_Jul', '2023_Jan', '2023_Jul', '2024_Jan', '2024_Jul', '2025_Jan']

# Define paths for each time point
file_paths = {
    '2022_Jul': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_july_2022.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_July_2022_Krishna.tif"
    },
    '2023_Jan': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_jan_2023.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_January_2023_Krishna.tif"
    },
    '2023_Jul': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_july_2023.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_July_2023_Krishna.tif"
    },
    '2024_Jan': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_jan_2024.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_01_January_2024_Krishna.tif"
    },
    '2024_Jul': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_july_2024.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_07_July_2024_Krishna.tif"
    },
    '2025_Jan': {
        'pred': r"/content/drive/MyDrive/predicted_ndvi_jan_2025.tif",
        'act':  r"/content/drive/MyDrive/NDVI_LSTM/NDVI_01_January_2025_Krishna.tif"
    }
}

# Define error threshold (in NDVI units)
threshold = 0.1

results = []

for key in time_keys:
    with rasterio.open(file_paths[key]['pred']) as src_pred:
        pred = src_pred.read(1)
    with rasterio.open(file_paths[key]['act']) as src_act:
        act = src_act.read(1)

    # Mask valid pixels
    mask = ~np.isnan(pred) & ~np.isnan(act)
    y_true = act[mask].flatten()
    y_pred = pred[mask].flatten()

    # Threshold-based accuracy
    abs_error = np.abs(y_pred - y_true)
    accurate_pixels = abs_error < threshold
    pixel_accuracy = np.sum(accurate_pixels) / len(y_true)

    # Store results
    year, month = key.split('_')
    results.append({
        'Year': year,
        'Month': month,
        'Accuracy': pixel_accuracy
    })

# Convert to DataFrame
df = pd.DataFrame(results)
table_acc = df.pivot(index='Year', columns='Month', values='Accuracy')
print(f"\nAccuracy Table for NDVI(Threshold = {threshold}):\n", table_acc.round(4))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Accuracy Table for NDVI(Threshold = 0.1):
 Month     Jan     Jul
Year                 
2022      NaN  0.4389
2023   0.3575  0.2616
2024   0.3061  0.3167
2025   0.3132     NaN
